In [1]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import sklearn.metrics as metrics
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
from keras.models import Sequential
import tensorflow as tf
from keras.layers import Dense
import keras.backend as K
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam
from keras.layers import Dropout
from keras.layers import LSTM, Dense
from keras.layers import GRU 
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense
def adj_r2_score(r2, n, k):
    return 1-((1-r2)*((n-1)/(n-k-1)))

In [2]:
new_model = tf.keras.models.load_model('saved_model/cnn_model')

In [3]:
new_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 6, 50)             10400     
                                                                 
 gru (GRU)                   (None, 6, 64)             22272     
                                                                 
 conv1d (Conv1D)             (None, 4, 64)             12352     
                                                                 
 max_pooling1d (MaxPooling1D  (None, 2, 64)            0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 2, 64)             0         
                                                                 
 flatten (Flatten)           (None, 128)               0         
                                                        

In [50]:
gbp_jpy_data = pd.read_csv('GBPJPY_H4.csv')
gold_data = pd.read_csv('GOLD_H4.csv')
us500_data = pd.read_csv('US500_H4.csv')
gbp_nzd_data = pd.read_csv('GBPNZD_H4.csv')
gbp_aud_data = pd.read_csv('GBPAUD_H4.csv')

list_of_df = []

list_of_df.append(gbp_jpy_data)
list_of_df.append(gold_data)
list_of_df.append(us500_data)
list_of_df.append(gbp_nzd_data)
list_of_df.append(gbp_aud_data)



In [51]:
index_list_of_df = []
for x in list_of_df:
    x["time"] = pd.to_datetime(x["time"])
    ind_exchange_data = x.set_index(["time"], drop=True)
    index_list_of_df.append(ind_exchange_data)
    
    

In [52]:

    
#Take a start date and end date
start_date = pd.Timestamp('31-12-10')
cut_off_date = pd.Timestamp('22-06-22')

clipped_data_list = []

for x in index_list_of_df:
    clipped_data = x.loc[start_date:cut_off_date]
    print(len(clipped_data))
    clipped_data_list.append(clipped_data)

17832
8794
4990
17832
17832


/var/folders/qs/tq01j0l577ddz8gg_tnvhl6m0000gn/T/ipykernel_11703/1250389640.py:8: FutureWarning: Indexing a timezone-aware DatetimeIndex with a timezone-naive datetime is deprecated and will raise KeyError in a future version. Use a timezone-aware object instead.
  clipped_data = x.loc[start_date:cut_off_date]


In [53]:
sc = StandardScaler()

In [54]:
for x in clipped_data_list: 
    df = x[["close"]]
    adjusted_data_frame = np.array(x)
    entries = int(len(x))
    split_length = int(entries * 0.8)
    split_date = x.index[split_length]
    
    train = df.loc[:split_date]
    test = df.loc[split_date:]
    
    train_sc = sc.fit_transform(train)
    test_sc = sc.transform(test)
    
    X = []
    y = []
    timesteps = 6
    for i in range(timesteps, train.shape[0]):
        X.append(train_sc[i-timesteps:i, 0])
        y.append(train_sc[i, 0])

    X = np.array(X)
    y = np.array(y)
    X = np.reshape(X, (X.shape[0], X.shape[1], 1))
    
    early_stop = EarlyStopping(monitor='loss', patience=5, verbose=1)
    history_model_lstm = new_model.fit(X, y, epochs=100, batch_size=192, verbose=1, shuffle=False, callbacks=[early_stop])
    
    

Epoch 1/100
75/75 [==============================] - 1s 8ms/step - loss: 0.0070 - rmse: 0.0839
Epoch 2/100
75/75 [==============================] - 1s 8ms/step - loss: 0.0086 - rmse: 0.0925
Epoch 3/100
75/75 [==============================] - 1s 8ms/step - loss: 0.0120 - rmse: 0.1097
Epoch 4/100
75/75 [==============================] - 1s 8ms/step - loss: 0.0144 - rmse: 0.1200
Epoch 5/100
75/75 [==============================] - 1s 8ms/step - loss: 0.0128 - rmse: 0.1130
Epoch 6/100
75/75 [==============================] - 1s 8ms/step - loss: 0.0139 - rmse: 0.1180
Epoch 6: early stopping
Epoch 1/100
37/37 [==============================] - 0s 8ms/step - loss: 0.0124 - rmse: 0.1112
Epoch 2/100
37/37 [==============================] - 0s 8ms/step - loss: 0.0135 - rmse: 0.1160
Epoch 3/100
37/37 [==============================] - 0s 8ms/step - loss: 0.0205 - rmse: 0.1432
Epoch 4/100
37/37 [==============================] - 0s 8ms/step - loss: 0.0163 - rmse: 0.1277
Epoch 5/100
37/37 [=======

In [55]:
def classify_numbers(lst):
    classifications = []
    prev_num = lst[0]
    for i, num in enumerate(lst[1:], start=1):
        if num > prev_num:
            classifications.append('green')
        else:
            classifications.append('red')
        prev_num = num
    return classifications



In [1]:
#Generate all the test data that is needed to make the predictions and display the predictions 

for count, x in enumerate(clipped_data_list): 
    df = x[["close"]]
    adjusted_data_frame = np.array(x)
    entries = int(len(x))
    split_length = int(entries * 0.8)
    split_date = x.index[split_length]
    
   
    test = df.loc[split_date:]
    test_sc = sc.transform(test)
    
    y_test = test_sc[1:]
    
    timesteps = 6
    X_test_lstm = []
    Y_test_lstm = []
    
    for i in range(timesteps, test.shape[0]):
        X_test_lstm.append(test_sc[i-timesteps:i, 0])
        Y_test_lstm.append(test_sc[i, 0])
    
    X_test_lstm = np.array(X_test_lstm)
    Y_test_lstm = np.array(Y_test_lstm)
    X_test_lstm = np.reshape(X_test_lstm, (X_test_lstm.shape[0], X_test_lstm.shape[1], 1))
    
    y_pred_test_lstm = new_model.predict(X_test_lstm)
    
    plt.figure(figsize=(12,8))
    plt.plot(y_test, label='True')
    plt.plot(y_pred_test_lstm, label='LSTM')
    plt.title("LSTM's_Prediction")
    plt.xlabel('Observation')
    plt.ylabel('INR_Scaled')
    plt.legend()
    plt.show()
    
    last_prediction = y_pred_test_lstm[-timesteps:]
    last_prediction = np.array(last_prediction)

    for i in range(10):
        next_input = np.array([last_prediction[-timesteps:]])
        next_input = next_input.reshape((next_input.shape[0], next_input.shape[1], 1))
        next_prediction = new_model.predict(next_input, verbose=0)
        last_prediction = np.append(last_prediction, next_prediction[0])
    

    lstm_future = np.array(last_prediction[-10:]).reshape(-1,1)
    inverse_transformed_data_lstm = sc.inverse_transform(lstm_future).flatten().tolist()   
    lstm_rounded_numbers = [round(x, 5) for x in inverse_transformed_data_lstm]
    
    future_lstm_prices = []
    last_lstm = sc.inverse_transform(y_pred_test_lstm[-1:])
    future_lstm_prices.append(round(float(last_lstm[0]),5))
    for z in lstm_rounded_numbers:
        future_lstm_prices.append(z) 
    
    clipped_len = len(x)
    future_comparison = index_list_of_df[count].iloc[clipped_len-1:]
    future = future_comparison.iloc[0:11]
    future_prices_table = future["close"]
    
    
    future_prices = []
    for z in future_prices_table:
        future_prices.append(z)
    
    print(future_prices)
        
    classifications1 = classify_numbers(future_lstm_prices)
    classifications2 = classify_numbers(future_prices)

    fig, (ax1, ax2) = plt.subplots(2, 1)

    ax1.plot(future_lstm_prices, color='blue')
    for i, c in enumerate(classifications1):
        if c == 'green':
            ax1.axvspan(i, i+1, facecolor='green', alpha=0.2)
        elif c == 'red':
            ax1.axvspan(i, i+1, facecolor='red', alpha=0.2)

    ax2.plot(future_prices, color='blue')
    for i, c in enumerate(classifications2):
        if c == 'green':
            ax2.axvspan(i, i+1, facecolor='green', alpha=0.2)
        elif c == 'red':
            ax2.axvspan(i, i+1, facecolor='red', alpha=0.2)

    plt.show()
    
    

    
    

NameError: name 'clipped_data_list' is not defined